In [4]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

In [5]:
products_data=pd.read_csv('balancedDATA.csv')
products_data

,NameDescription,categories
0,surco a conversion bar bike rack featuresused ...,Automotive
1,kent marine turbo calcium ounce bottle turbo ...,Pet Supplies
2,inch pro style swim snorkel breathe underwater...,Sports & Outdoors
3,zia natural skincare seaweed lift serum ounce ...,Beauty
4,nexcare waterproof bandage knee amp elbow ct ...,Health & Personal Care
...,...,...
35761,palladio rice paper natural unique tissue made...,"Arts, Crafts & Sewing"
35762,learning resource tool set piece molded plasti...,Toys & Games
35763,serious skin care aeye xr nanoencapsulated eye...,Beauty
35764,wireless phone xcessories travel charger samsu...,Cell Phones & Accessories


In [6]:
products_data['categories'].unique().shape

(18,)

# Extracting Features From TextData using TF-IDF

In [7]:
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing

In [8]:
Text=[]
for text in products_data["NameDescription"]:
    Text.append(text)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
# First, we need a vectorizer object
vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(Text).toarray()
xx=  pd.DataFrame(x)
xx

,0,1,2,3,4,5,6,7,8,9,...,99060,99061,99062,99063,99064,99065,99066,99067,99068,99069
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35761,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35762,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35763,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35764,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


After feature extraction from our text it gives 99070 features.

# Label 

In [12]:
l=[]
for text in products_data["categories"].unique():
    l.append(text)

In [14]:

y = []

for text in products_data["categories"]:
    for j in range(18):
        if text==l[j]:
            y.append(j)
    
y = np.asarray(y)

yy = pd.Series(y)
yy

0        0
1        1
2        2
3        3
4        4
        ..
35761    5
35762    7
35763    3
35764    6
35765    7
Length: 35766, dtype: int64

### We will trying to find the best peurcentage of features that can we use and give us more best accuraccy

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=1,test_size=0.2)

# MLP after feature selection

In [16]:
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.metrics import accuracy_score
import pickle
from sklearn.neural_network import MLPClassifier


def select_features_CH2(x_train, y_train, x_test):
    #CH2 = SelectKBest(score_func=chi2, k=3390)
    CH2 = SelectPercentile(chi2, percentile=15)
    CH2.fit(x_train, y_train)
    x_train_CH2 = CH2.transform(x_train)
    x_test_CH2 = CH2.transform(x_test)
    return x_train_CH2, x_test_CH2, CH2

accuracies = {}

for p in range(5, 100, 5):
    print("Percentage of features is:", p)
    x_train_CH2, x_test_CH2, CH2 = select_features_CH2(x_train, y_train, x_test)
    n = x_train_CH2.shape[1]
    print("Number of features is:", n)
    classifier = MLPClassifier(hidden_layer_sizes=(250,100,50), max_iter=10, verbose=True)
    classifier.fit(x_train_CH2, y_train)
    y_pred = classifier.predict(x_test_CH2)
    accuracy = round(accuracy_score(y_pred,y_test)*100,2)
    print("Accuracy for the {} best features is: {}%".format(n, accuracy))
    ss = "p-"+str(p)+"_n-"+str(n)+"_acc-"+str(accuracy)+".pickle"
    pickle.dump(classifier, open(ss, "wb"))
    accuracies[p] = [n, accuracy, ss]

Percentage of features is: 5
Number of features is: 14861
Iteration 1, loss = 1.68062593
Iteration 2, loss = 0.39035010
Iteration 3, loss = 0.18358800
Iteration 4, loss = 0.09562308
Iteration 5, loss = 0.05125799
Iteration 6, loss = 0.03210136
Iteration 7, loss = 0.02455687
Iteration 8, loss = 0.01841269
Iteration 9, loss = 0.01591150
Iteration 10, loss = 0.01312379
Accuracy for the 14861 best features is: 84.08%
Percentage of features is: 10
Number of features is: 14861
Iteration 1, loss = 1.64611069
Iteration 2, loss = 0.39726251
Iteration 3, loss = 0.19394478
Iteration 4, loss = 0.10031732
Iteration 5, loss = 0.05511021
Iteration 6, loss = 0.03537237
Iteration 7, loss = 0.02311817
Iteration 8, loss = 0.02026911
Iteration 9, loss = 0.01574324
Iteration 10, loss = 0.01739208
Accuracy for the 14861 best features is: 83.24%
Percentage of features is: 15
Number of features is: 14861
Iteration 1, loss = 1.62549595
Iteration 2, loss = 0.37695601
Iteration 3, loss = 0.18246437
Iteration 4, 